In [3]:
from scipy import stats
import pandas as pd
import numpy as np

In [48]:
X = pd.DataFrame(dict(a=[1.0, 1.2, 0.9, 0.98, 12.9, 12.9, 12.5], b=[1.0, 1.2, 0.9, 0.98, 12.9, 12.9, 12.9], c=[1.0, 0.9, 1.2, 0.98, 12.9, 12.9, 12.9]))
X 

,a,b,c
0,1.00,1.00,1.00
1,1.20,1.20,0.90
2,0.90,0.90,1.20
3,0.98,0.98,0.98
4,12.90,12.90,12.90
5,12.90,12.90,12.90
6,12.50,12.90,12.90


In [90]:
Y = pd.DataFrame(dict(a=[1789.0, 110.2, 0.29, 01.98, 1200.9, 11200.9, 120002.1], b=[1000.0, 100.2, 0.009, 30000.98, 10002.9, 12.9, 12.9], c=[143.0, 04.9, 165.2, 0.98, 120.9, 1782.9, 182.9]))
Y

,a,b,c
0,1789.00,1000.000,143.00
1,110.20,100.200,4.90
2,0.29,0.009,165.20
3,1.98,30000.980,0.98
4,1200.90,10002.900,120.90
5,11200.90,12.900,1782.90
6,120002.10,12.900,182.90


In [92]:
stats.wilcoxon(X, Y, axis=None)

WilcoxonResult(statistic=np.float64(3.0), pvalue=np.float64(0.0003270126629328652))

In [86]:
stats.wilcoxon((1,2,3,4,5,-1), (700.0,3,2,5,6,4))

WilcoxonResult(statistic=np.float64(2.5), pvalue=np.float64(0.15625))

In [39]:
type((1,(1),2,3))

tuple

In [31]:
Y = pd.DataFrame(dict(a=[1.0, 1.2, 0.9, 0.98, 12.9, 12.9, 12.5]))
Y

,a
0,1.00
1,1.20
2,0.90
3,0.98
4,12.90
5,12.90
6,12.50


In [93]:
pd.read_csv(, sep="/t")

NameError: name 'sep' is not defined